In [ ]:
import numpy as np
import pandas as pd
import os
os.environ["KERAS_BACKEND"] = "pytorch"
import keras
from keras import layers
!pip install rdkit-pypi==2023.3.1b1 --upgrade
from rdkit import Chem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 9.6 MB/s eta 0:00:00


In [ ]:
!mkdir ~/.kaggle
!echo '{"username":"michaelmoser1","key":"fb607e421aa0b4036f30d872f4f26f92"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

In [ ]:
! kaggle competitions download leash-BELKA

100% 4.14G/4.16G [00:34<00:00, 119MB/s]
100% 4.16G/4.16G [00:34<00:00, 131MB/s]


In [ ]:
! unzip leash-BELKA.zip

Archive:  leash-BELKA.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test.parquet            
  inflating: train.csv               
  inflating: train.parquet           


In [ ]:
def X_processing(pca, X):
  pca.fit(X)
  return pca.transform(X)

In [ ]:
from rdkit.Chem import Descriptors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# chunk_size = 100
# df = pd.read_csv('train.csv', chunksize=chunk_size)
# i = 0
# for chunk in df:
#   descriptors = [Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(mol)) for mol in chunk['molecule_smiles'].tolist()]
#   features_df = pd.DataFrame(descriptors)
#   X = features_df.to_numpy()
#   Y =
#   pca = PCA(n_components=209)
#   pca.fit(X)
#   break

import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_dim=30),
    Dense(64, activation='relu'),
    Dense(3, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

chunk_size = 100000
df = pd.read_csv('train.csv', chunksize=chunk_size)

descriptor_names = [desc[0] for desc in Descriptors.descList]

# Assuming df is defined earlier as a pandas DataFrame iterator
# e.g., df = pd.read_csv('train.csv', chunksize=chunk_size)
for chunk in df:
  pivot = chunk.pivot(index='molecule_smiles', columns='protein_name', values='binds')
  pivot.reset_index(inplace=True)
  pivot['Descriptors'] = pivot['molecule_smiles'].apply(lambda mol: [desc[1](Chem.MolFromSmiles(mol)) for desc in Descriptors.descList])

  descriptors_df = pd.DataFrame(pivot['Descriptors'].tolist(), columns=descriptor_names, index=pivot.index)
  final_df = pd.concat([pivot, descriptors_df], axis=1)
  X = final_df.drop(columns=['molecule_smiles'] + list(pivot.columns.get_level_values(0))).to_numpy()
  Y = pivot[['BRD4', 'HSA', 'sEH']].to_numpy()

  X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
  Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)

  scaler = StandardScaler()
  X = scaler.fit_transform(X)
  pca = PCA(n_components=30)
  Theta_train = X_processing(pca, X)

  model.fit(Theta_train, Y, epochs=10, batch_size=32, validation_split=0.2)

  break


Number of features for 85% energy: 33


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rdkit import Chem
from rdkit.Chem import Descriptors

model = Sequential([
    Dense(128, activation='relu', input_dim=30),
    Dense(64, activation='relu'),
    Dense(3, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

chunk_size = 100000
train_file = 'train.csv'
descriptor_names = [desc[0] for desc in Descriptors.descList]

def X_processing(pca, X):
    return pca.transform(X)

df_iter = pd.read_csv(train_file, chunksize=chunk_size)
initial_chunk = next(df_iter)

pivot = initial_chunk.pivot(index='molecule_smiles', columns='protein_name', values='binds')
pivot.reset_index(inplace=True)
pivot['Descriptors'] = pivot['molecule_smiles'].apply(lambda mol: [desc[1](Chem.MolFromSmiles(mol)) for desc in Descriptors.descList])

In [ ]:
descriptors_df = pd.DataFrame(pivot['Descriptors'].tolist(), columns=descriptor_names, index=pivot.index)
final_df = pd.concat([pivot, descriptors_df], axis=1)

X_initial = final_df.drop(columns=['molecule_smiles'] + list(pivot.columns.get_level_values(0))).to_numpy()

X_initial = np.nan_to_num(X_initial, nan=0.0, posinf=1e9, neginf=-1e9)

scaler = StandardScaler().fit(X_initial)
pca = PCA(n_components=30).fit(scaler.transform(X_initial))

i = 0
for chunk in pd.read_csv(train_file, chunksize=chunk_size):
    print(i)
    pivot = chunk.pivot(index='molecule_smiles', columns='protein_name', values='binds')
    pivot.reset_index(inplace=True)
    pivot['Descriptors'] = pivot['molecule_smiles'].apply(lambda mol: [desc[1](Chem.MolFromSmiles(mol)) for desc in Descriptors.descList])

    descriptors_df = pd.DataFrame(pivot['Descriptors'].tolist(), columns=descriptor_names, index=pivot.index)
    final_df = pd.concat([pivot, descriptors_df], axis=1)

    X = final_df.drop(columns=['molecule_smiles'] + list(pivot.columns.get_level_values(0))).to_numpy()
    Y = pivot[['BRD4', 'HSA', 'sEH']].to_numpy()

    X = np.nan_to_num(X, nan=0.0, posinf=1e9, neginf=-1e9)
    Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)

    X = scaler.transform(X)
    Theta_train = X_processing(pca, X)

    model.fit(Theta_train, Y, epochs=10, batch_size=32, validation_split=0.2)

    i += 1

model.save('trained_model.h5')

0
Epoch 1/10
834/834 [==============================] - 6s 3ms/step - loss: 0.0218 - accuracy: 0.3581 - val_loss: 0.0129 - val_accuracy: 0.2553
Epoch 2/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0131 - accuracy: 0.3397 - val_loss: 0.0130 - val_accuracy: 0.2811
Epoch 3/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0123 - accuracy: 0.3239 - val_loss: 0.0142 - val_accuracy: 0.3300
Epoch 4/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0112 - accuracy: 0.2739 - val_loss: 0.0148 - val_accuracy: 0.3654
Epoch 5/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0102 - accuracy: 0.2884 - val_loss: 0.0150 - val_accuracy: 0.3150
Epoch 6/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0094 - accuracy: 0.2847 - val_loss: 0.0171 - val_accuracy: 0.5823
Epoch 7/10
834/834 [==============================] - 3s 3ms/step - loss: 0.0086 - accuracy: 0.2717 - val_loss: 0.0163 - val_accuracy: 0.5535
Epoc

In [ ]:
test_pivot = test_df.pivot(index='molecule_smiles', columns='protein_name', values='binds')
test_pivot.reset_index(inplace=True)
test_pivot['Descriptors'] = test_pivot['molecule_smiles'].apply(lambda mol: [desc[1](Chem.MolFromSmiles(mol)) for desc in Descriptors.descList])

test_descriptors_df = pd.DataFrame(test_pivot['Descriptors'].tolist(), columns=descriptor_names, index=test_pivot.index)
test_final_df = pd.concat([test_pivot, test_descriptors_df], axis=1)

X_test = test_final_df.drop(columns=['molecule_smiles'] + list(test_pivot.columns.get_level_values(0))).to_numpy()
X_test = np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)

X_test = scaler.transform(X_test)
Theta_test = X_processing(pca, X_test)


predictions = model.predict(Theta_test)

NameError: name 'test_df' is not defined